In [1]:
pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 28.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Connect to Cassandra
auth_provider = PlainTextAuthProvider(username='admin', password='admin')
cluster = Cluster(['cassandra'], auth_provider=auth_provider)
session = cluster.connect()

# Select your keyspace


create_database_cql = "CREATE KEYSPACE demo WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};"
session.execute(create_database_cql)



create_result_table_cql="""
CREATE TABLE IF NOT EXISTS latest_flight_data (
    callsign TEXT PRIMARY KEY,
    longitude DOUBLE,
    latitude DOUBLE,
    on_ground BOOLEAN,
    squawk TEXT
);
"""
session.execute(create_result_table_cql)



AlreadyExists: Keyspace 'demo' already exists